In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.ryerson.ca/calendar/2021-2022/programs/feas/mechanical/"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
#open all the tabs first in order to click the links
open_tabs = driver.find_element_by_class_name("resAccordionOpenAll")
open_tabs.click()
time.sleep(2)

In [7]:
mechatronics_container = page_soup.findAll("div", {"class": "panel panel-default"})[4]

In [9]:
#need to filter out a liberal studies courses container
sub_containers = mechatronics_container.findAll("div", {"class": "resTable parbase section"})
len(sub_containers)

7

In [10]:
sub_containers[3]

<div class="resTable parbase section">
<div class="scrollIndicator">
<i aria-hidden="true" class="fa fa-long-arrow-left"></i> Swipe to see more <i aria-hidden="true" class="fa fa-long-arrow-right"></i>
</div>
<div id="5256fafe-8891-4392-95a9-27f77e2bdeec">
<table border="0" cellpadding="0" cellspacing="0" class="table table-bordered">
<colgroup>
<col width="20%"/>
<col width="80%"/>
</colgroup>
<tbody>
<tr>
<td valign="top"><a class="qTipCourse" data-url="/content/ryerson/calendar/2021-2022/courses/english/ENG/503/jcr:content/resCalendarCourseEmbed.json" href="/content/ryerson/calendar/2021-2022/courses/english/ENG/503.html">ENG 503</a></td>
<td valign="top">Science Fiction</td>
</tr>
<tr>
<td valign="top"><a class="qTipCourse" data-url="/content/ryerson/calendar/2021-2022/courses/geography/GEO/702/jcr:content/resCalendarCourseEmbed.json" href="/content/ryerson/calendar/2021-2022/courses/geography/GEO/702.html">GEO 702</a></td>
<td valign="top">Technology and the Contemporary Environme

In [11]:
sub_containers = [container for container in sub_containers if sub_containers.index(container) != 3]
len(sub_containers)

6

In [12]:
link_texts = [container.findAll("a", {"class": "qTipCourse"}) for container in sub_containers]
link_texts = [link_text.text for sub_list in link_texts for link_text in sub_list]
link_texts

['EES 508',
 'EES 612',
 'MEC 430',
 'MEC 514',
 'MEC 516',
 'MTH 510',
 'MEC 411',
 'MEC 613',
 'MEC 701',
 'MEC 709',
 'MEC 721',
 'MEC 733',
 'MEC 751',
 'MEC 830',
 'ELE 719',
 'MEC 617',
 'MEC 626',
 'MEC 713',
 'MEC 813',
 'CEN 800',
 'ELE 709',
 'MEC 825',
 'ELE 819',
 'ELE 888',
 'MEC 809',
 'MEC 816',
 'MEC 832']

In [13]:
len(link_texts)

27

# 2. Automation script to scrape all courses

In [14]:
#scrape the upper years courses

course_codes = []
course_names = []
course_descs = []
counter = 0

driver.get("https://www.ryerson.ca/calendar/2021-2022/programs/feas/mechanical/")
time.sleep(3)
driver.find_element_by_class_name("resAccordionOpenAll").click()
time.sleep(3)

for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()    
    time.sleep(2)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("div", {"class": "resCalendarCourseEmbed resCalendarCourse"})
    course_code = container.find("h1").text.strip()
    course_name = container.find("h2").text.strip()
    course_desc = container.find("div", {"class": "courseDescription"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(2)
    driver.find_element_by_class_name("resAccordionOpenAll").click()
    time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  EES 508
Scraped  EES 612
Scraped  MEC 430
Scraped  MEC 514
Scraped  MEC 516
Scraped  MTH 510
Scraped  MEC 411
Scraped  MEC 613
Scraped  MEC 701
Scraped  MEC 709
Scraped  MEC 721
Scraped  MEC 733
Scraped  MEC 751
Scraped  MEC 830
Scraped  ELE 719
Scraped  MEC 617
Scraped  MEC 626
Scraped  MEC 713
Scraped  MEC 813
Scraped  CEN 800
Scraped  ELE 709
Scraped  MEC 825
Scraped  ELE 819
Scraped  ELE 888
Scraped  MEC 809
Scraped  MEC 816
Scraped  MEC 832
Finished scraping 27 courses


In [15]:
course_codes

['EES 508',
 'EES 612',
 'MEC 430',
 'MEC 514',
 'MEC 516',
 'MTH 510',
 'MEC 411',
 'MEC 613',
 'MEC 701',
 'MEC 709',
 'MEC 721',
 'MEC 733',
 'MEC 751',
 'MEC 830',
 'ELE 719',
 'MEC 617',
 'MEC 626',
 'MEC 713',
 'MEC 813',
 'CEN 800',
 'ELE 709',
 'MEC 825',
 'ELE 819',
 'ELE 888',
 'MEC 809',
 'MEC 816',
 'MEC 832']

In [16]:
course_names

['Digital Systems',
 'Electric Machines and Actuators',
 'Solid Mechanics I',
 'Applied Thermodynamics',
 'Fluid Mechanics I',
 'Numerical Analysis',
 'Mechanics of Machines',
 'Machine Design I',
 'Heat Transfer',
 'Control Systems',
 'Vibrations',
 'Microprocessor Systems',
 'Measurements, Sensors and Instruments',
 'Mechatronics Systems Design',
 'Fundamentals of Robotics',
 'Manufacturing System Control',
 'Applied Finite Elements',
 'Project Management',
 'Flexible Manufacturing Systems',
 'Law and Ethics in Engineering Practice',
 'Real-Time Computer Control Systems',
 'Mechanical Design',
 'Control of Robotic Manipulators',
 'Intelligent Systems',
 'Integrated Manufacturing',
 'Topics in Manufacturing Engineering',
 'Reliability and Decision Analysis']

In [17]:
course_descs

['Number systems, codes and coding, minimization techniques applied to design of logic systems. Component specifications. Discussion of microprocessors, memory and I/O logic elements. Microcomputer structure and operation. I/O modes and interfacing. Machine language and Assembler programming. Design and application of digital systems for data collection and control of pneumatic hydraulic and machine systems. Laboratory work includes the use of microcomputers.',
 'The single-phase transformer and its applications. DC and AC motor characteristics, and their application in mechanical drives. Power electronic circuits, H bridges, PWM control, interfacing, power amplifiers. DC servo and stepper motors, AC synchronous and induction motors. Transformers. Introduction to typical speed and torque control techniques of motors.',
 "Stresses and deflections; statically indeterminate problems. Stress transformation; principal stresses; Mohr's Circle; theories of failure. Flexural analysis; the meth

In [18]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,EES 508,Digital Systems,"Number systems, codes and coding, minimization..."
1,EES 612,Electric Machines and Actuators,The single-phase transformer and its applicati...
2,MEC 430,Solid Mechanics I,Stresses and deflections; statically indetermi...
3,MEC 514,Applied Thermodynamics,"Rankine cycle, steam turbine cycles, reheat an..."
4,MEC 516,Fluid Mechanics I,"Dimensions and units, continuum fluid mechanic..."
5,MTH 510,Numerical Analysis,"Review of Taylor's formula, truncation error a..."
6,MEC 411,Mechanics of Machines,"Dynamics of complex, multi-component systems; ..."
7,MEC 613,Machine Design I,Review of stress analysis procedures; combined...
8,MEC 701,Heat Transfer,A fundamental course in heat transfer includin...
9,MEC 709,Control Systems,Mathematical model representation of physical ...


In [19]:
df.to_csv('Ryerson_MechEng_Mechatronics_Option_Core_and_Electives_(Years 3-4)_Courses.csv', index = False)

In [20]:
driver.quit()